In [ ]:
from keras.layers import *
from keras.models import Model
from keras import layers
from keras.layers.merge import concatenate
import sys
sys.path.insert(1, '../src')
sys.path.insert(1, '../image_segmentation_keras')
from keras_segmentation.models.config import IMAGE_ORDERING

from keras_segmentation.models.model_utils import get_segmentation_model
from glob import glob
from crfrnn_layer import CrfRnnLayer

In [ ]:
input_height = 256
input_width = 256
n_classes = 3
channels = 3

In [ ]:
def unet_conv_block(inputs, filters, pool=True, batch_norm_first=True):
    if batch_norm_first == True:
        x = Conv2D(filters, 3, padding="same")(inputs)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)

        x = Conv2D(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
    elif batch_norm_first == False:
        x = Conv2D(filters, 3, padding="same")(inputs)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)

        x = Conv2D(filters, 3, padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)

    if pool == True:
        p = MaxPooling2D((2, 2))(x)
        return [x, p]
    else:
        return x

In [ ]:
img_input = Input(shape=(input_height,input_width, channels))
f1 = unet_conv_block(img_input, 64, pool=True, batch_norm_first=True)
f2 = unet_conv_block(f1[1], 128, pool=True, batch_norm_first=True)
f3 = unet_conv_block(f2[1], 256, pool=True, batch_norm_first=True)
f4 = unet_conv_block(f3[1], 512, pool=True, batch_norm_first=True)
f5 = unet_conv_block(f4[1], 1024, pool=False, batch_norm_first=True)

x = UpSampling2D((2, 2))(f5)
x = concatenate([x, f4[0]], axis=3)
x = unet_conv_block(x, 512, pool=False, batch_norm_first=True)

x = UpSampling2D((2, 2))(x)
x = concatenate([x, f3[0]], axis=3)
x = unet_conv_block(x, 256, pool=False, batch_norm_first=True)

x = UpSampling2D((2, 2))(x)
x = concatenate([x, f2[0]], axis=3)
x = unet_conv_block(x, 128, pool=False, batch_norm_first=True)

x = UpSampling2D((2, 2))(x)
x = concatenate([x, f1[0]], axis=3)
x = unet_conv_block(x, 64, pool=False, batch_norm_first=True)

x = Conv2D(n_classes, (3, 3), padding='same')(x)

model = get_segmentation_model(img_input, x)


In [18]:
model.train(
    train_images =  "/Users/mavaylon/Research/image-segmentation-keras/Data/train/img/",
    train_annotations = "/Users/mavaylon/Research/image-segmentation-keras/Data/train/ann/",
    epochs=20,
    steps_per_epoch=len(glob("/Users/mavaylon/Research/image-segmentation-keras/Data/train/img/*")),
    batch_size=1,
    checkpoints_path="/Users/mavaylon/Research/LBNL_Segmentation_crf/unet_pet_class.h5",
    auto_resume_checkpoint=True,
    validate=True,
    val_images="/Users/mavaylon/Research/image-segmentation-keras/Data/test/img/",
    val_annotations="/Users/mavaylon/Research/image-segmentation-keras/Data/test/ann/",
    val_batch_size=1,
    val_steps_per_epoch=len(glob("/Users/mavaylon/Research/image-segmentation-keras/Data/test/img/*"))
)

  0%|          | 0/5912 [00:00<?, ?it/s]

Verifying training dataset


  2%|▏         | 31/1478 [00:00<00:04, 303.25it/s]

Dataset verified! 
Verifying validation dataset


100%|██████████| 1478/1478 [00:05<00:00, 293.55it/s]


Dataset verified! 
correct
Epoch 1/20
5912/5912 [==============================] - 11408s 2s/step - loss: 0.1800 - accuracy: 0.9311 - val_loss: 0.1774 - val_accuracy: 0.8749

Epoch 00001: val_accuracy improved from -inf to 0.87492, saving model to pet_class_crf.h5
Epoch 2/20
5912/5912 [==============================] - 11871s 2s/step - loss: 0.1651 - accuracy: 0.9363 - val_loss: 0.1788 - val_accuracy: 0.8762

Epoch 00002: val_accuracy improved from 0.87492 to 0.87618, saving model to pet_class_crf.h5
Epoch 3/20
5912/5912 [==============================] - 12340s 2s/step - loss: 0.1536 - accuracy: 0.9406 - val_loss: 0.1601 - val_accuracy: 0.8773

Epoch 00003: val_accuracy improved from 0.87618 to 0.87727, saving model to pet_class_crf.h5
Epoch 4/20
5912/5912 [==============================] - 11207s 2s/step - loss: 0.1484 - accuracy: 0.9423 - val_loss: 0.1595 - val_accuracy: 0.8758

Epoch 00004: val_accuracy did not improve from 0.87727
Epoch 5/20
5912/5912 [============================

KeyboardInterrupt: 